In [66]:
from glob import glob
import numpy as np
import cv2
from gi.repository import Vips
import matplotlib
from matplotlib import pyplot as plt
from skimage import exposure
from scipy.misc import bytescale
import pvl
import os.path
import numpy

%matplotlib inline

In [12]:
def imshow(img):
    if len(img.shape) == 2 or img.shape[2] == 1:
        img = np.dstack([img, img, img])
    b, g, r = cv2.split(img)
    rgb_img = cv2.merge([r, g, b])
    plt.imshow(rgb_img)
    plt.show()

In [13]:
def read_vips(filename):
    return Vips.Image.new_from_file(filename)

def extract_gray(image, offset_x, offset_y, size_x=256, size_y=256):
    subimage = image.extract_area(offset_x, offset_y, size_x, size_y)
    mat = np.fromstring(subimage.write_to_memory()).view(np.int16)
    return np.uint8(mat.reshape(size_y, size_x) / 8)

def extract_bgr(image, offset_x, offset_y, size_x=256, size_y=256):
    subimage = image.extract_area(offset_x, offset_y, size_x, size_y)
    channels = [1, 2, 4]
    mat = np.fromstring(subimage.write_to_memory()).view(np.int16)
    return np.uint8(mat.reshape(size_y, size_x, 8) / 8)[:, :, channels]
    
def rescale_bands(image):
    scaled_image = np.zeros(image.shape, dtype=np.float)
    q = (2, 98)
    for i in range(3):
        p1, p2 = np.percentile(image[:, :, i], q)
        scaled_image[:, :, i] = exposure.rescale_intensity(image[:, :, i], in_range=(p1, p2))
    return bytescale(scaled_image)

In [16]:
dir_path = "./data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_PAN/"


files = sorted(glob(dir_path + '*.TIF'))
print(files)

img0_vips = read_vips(files[0])
img1_vips = read_vips(files[1])
tile0 = extract_rgb(img0_vips, 1024, 1024, 1024, 1024)
tile0 = rescale_bands(tile0)
tile1 = extract_rgb(img1_vips, 1024, 1024, 1024, 1024)
tile1 = rescale_bands(tile1)

imshow(tile0)
imshow(tile1)

cv2.imwrite(dir_path + "/0.png", tile0)
cv2.imwrite(dir_path + "/1.png", tile1)

['./data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063658-M2AS_R1C1-053950035020_01_P001.TIF', './data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063658-M2AS_R1C2-053950035020_01_P001.TIF', './data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063658-M2AS_R2C1-053950035020_01_P001.TIF', './data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063658-M2AS_R2C2-053950035020_01_P001.TIF', './data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063756-M2AS_R1C1-053950035020_01_P001.TIF', './data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063756-M2AS_R1C2-053950035020_01_P001.TIF', './data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063756-M2AS_R2C1-053950035020_01_P001.TIF', './data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/14OCT06063756-M2AS

Error: Error calling operator extract_area.
  extract_area: bad extract area


In [95]:
dir_path = "./data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_PAN/"
output_path = "./data/output/"

def mercator_to_pixel(val):
    return round(val / 0.4 - 0.5)

def read_or_create(x, y):
    global output_path
    filename = output_path + str(x) + '_' + str(y) + '.png'
    
    if os.path.exists(filename):
        return cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    else:
        return numpy.zeros((256, 256, 4))
    
def write(x, y, img):
    global output_path
    filename = output_path + str(x) + '_' + str(y) + '.png'
    cv2.imwrite(filename, img)

def process_tif(info, lod):
    _EARTH_RADIUS_M = 6378137.0
    _WMTS_WIDTH = 2.0 * np.pi * _EARTH_RADIUS_M
    _WMTS_OFFSET = -0.5 * _WMTS_WIDTH

    global dir_path
    image = Vips.Image.new_from_file(dir_path + info['filename'])
    
    img_llx = mercator_to_pixel(info['LLX'])
    img_lly = mercator_to_pixel(info['LLY'])
    
    img_urx = mercator_to_pixel(info['URX']) + 1
    img_ury = mercator_to_pixel(info['URY']) + 1
    
    tile_num_from_x = img_llx // 256
    tile_num_from_y = img_lly // 256    

    tile_num_to_x = (img_urx + 255) // 256
    tile_num_to_y = (img_ury + 255) // 256
    
    print(tile_num_from_x)
    
    for tile_x in range(tile_num_from_x, tile_num_to_x):
        for tile_y in range(tile_num_from_y, tile_num_to_y):
            tile = read_or_create(tile_x, tile_y)
            print(tile.shape)
            left_x = tile_x * 256 - img_llx
            left_y = tile_y * 256 - img_lly
            
            offset_x = max(0, -left_x)
            offset_y = max(0, -left_y)
            
            x_range = min(256, img_urx - img_llx - left_x)
            y_range = min(256, img_ury - img_lly - left_y)
            
            cur_img = extract_gray(image, left_x + offset_x, left_y + offset_y, x_range - offset_x, y_range - offset_y)
  
            print(tile.shape)
            print(cur_img.shape)
            tile[offset_y:y_range, offset_x:x_range, 0] = cur_img[0:, 0:]
            tile[offset_y:y_range, offset_x:x_range, 1] = cur_img[0:, 0:]
            tile[offset_y:y_range, offset_x:x_range, 2] = cur_img[0:, 0:]
            tile[offset_y:y_range, offset_x:x_range, 3] = 255
            
            write(tile_x, tile_y, tile)

info = pvl.load(dir_path + '14OCT06063756-P2AS-053950035020_01_P001.TIL')

for i in range(info['numTiles']):
    filename = info['TILE_' + str(i + 1)]['filename']
    process_tif(info['TILE_' + str(i + 1)])

6103
!
(256, 256, 4)
1570688 6537000
-64 -148 256 256
(256, 256, 4)
(108, 192)
!
(256, 256, 4)
1570688 6536744
-64 108 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6536488
-64 364 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6536232
-64 620 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6535976
-64 876 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6535720
-64 1132 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6535464
-64 1388 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6535208
-64 1644 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6534952
-64 1900 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6534696
-64 2156 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6534440
-64 2412 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6534184
-64 2668 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6533928
-64 2924 256 256
(256, 256, 4)
(256, 192)
!
(256, 256, 4)
1570688 6533672
-64 3